# `bsym` – a basic symmetry module

`bsym` is a basic Python symmetry module. The module consists of some core classes that describe the symmetry properties and arrangements of objects, and functions for performing symmetry-dependent operations. 

API documentation is [here](http://bsym.readthedocs.io).

## Overview

The central object described by `bsym` is the **configuration space**. This defines a vector space that can be occupied by other objects. For example; the three points $a, b, c$ defined by an equilateral triangle,

<img src='figures/triangular_configuration_space.pdf'>

which can be described by a length 3 vector:

\begin{pmatrix}a\\b\\c\end{pmatrix}

If these points can be coloured black or white, then we can define a **configuration** for each different colouring (0 for white, 1 for black), e.g. 

<img src='figures/triangular_configuration_example_1.pdf'>

with the corresponding vector

\begin{pmatrix}1\\1\\0\end{pmatrix}

A specific **configuration** therefore defines how objects are distributed within a particular **configuration space**.

The symmetry relationships between the different vectors in a **configuration space** are described by **symmetry operations**. A **symmetry operation** describes a transformation of a **configuration space** that leaves it indistinguishable. Each **symmetry operation** can be describes as a matrix that maps the vectors in a **configuration space** onto each other, e.g. in the case of the equiateral triangle the simplest **symmetry operation** is the identity, $E$, which leaves every corner unchanged, and can be represented by the matrix 

\begin{equation}
E=\begin{pmatrix}1 & 0 & 0\\0 & 1 & 0 \\ 0 & 0 & 1\end{pmatrix}
\end{equation}

For this triangular example, there are other **symmetry operations**, including reflections, $\sigma$ and rotations, $C_n$:

<img src='figures/triangular_example_symmetry_operations.pdf'>

In this example reflection operation, $b$ is mapped to $c$; $b\to c$, and $c$ is mapped to $b$; $b\to c$. 

The matrix representation of this **symmetry operation** is

\begin{equation}
\sigma_\mathrm{a}=\begin{pmatrix}1 & 0 & 0\\0 & 0 & 1 \\ 0 & 1 & 0\end{pmatrix}
\end{equation}

For the example rotation operation, $a\to b$, $b\to c$, and $c\to a$, with matrix representation

\begin{equation}
C_3=\begin{pmatrix}0 & 0 & 1\\ 1 & 0 & 0 \\ 0 & 1 & 0\end{pmatrix}
\end{equation}

Using this matrix and vector notation, the effect of a symmetry operation on a specific **configuration** can be calculated as the [matrix product](https://en.wikipedia.org/wiki/Matrix_multiplication#Square_matrix_and_column_vector) of the **symmetry operation** matrix and the **configuration** vector:

<img src='figures/triangular_rotation_operation.pdf'>

In matrix notation this is represented as

\begin{equation}
\begin{pmatrix}0\\1\\1\end{pmatrix} = \begin{pmatrix}0 & 0 & 1\\ 1 & 0 & 0 \\ 0 & 1 & 
0\end{pmatrix}\begin{pmatrix}1\\1\\0\end{pmatrix}
\end{equation}

or more compactly

\begin{equation}
c_\mathrm{f} = C_3 c_\mathrm{i}.
\end{equation}

The set of all symmetry operations for a particular **configuration space** is a **group**. 

For an equilateral triangle this group is the $C_{3v}$ [point group](https://en.wikipedia.org/wiki/Point_group), which contains six symmetry operations: the identity, three reflections (each with a mirror plane bisecting the triangle and passing through $a$, $b$, or $c$ respectively) and two rotations (120° clockwise and counterclockwise).

\begin{equation}
C_{3v} = \left\{ E, \sigma_\mathrm{a}, \sigma_\mathrm{b}, \sigma_\mathrm{c}, C_3, C_3^\prime \right\}
\end{equation}


## The SymmetryOperation class

A **symmetry operation** is represented by an instance of the `SymmetryOperation` class.

In [1]:
from bsym import SymmetryOperation

Create a `SymmetryOperation` object for `E`.

In [2]:
SymmetryOperation([[ 1, 0, 0 ], 
                   [ 0, 1, 0 ], 
                   [ 0, 0, 1 ]])

SymmetryOperation
label(---)
matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

A `SymmetryOperation` object records the matrix representation of the **symmetry operation** and an optional label. We can provide the label during creation:

In [3]:
SymmetryOperation([[ 1, 0, 0 ], 
                   [ 0, 1, 0 ], 
                   [ 0, 0, 1 ]], label='E' )

SymmetryOperation
label(E)
matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

or set it afterwards:

In [4]:
e = SymmetryOperation([[ 1, 0, 0 ], 
                       [ 0, 1, 0 ], 
                       [ 0, 0, 1 ]])
e.label = 'E'
e

SymmetryOperation
label(E)
matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

TODO: explain vector representation of symmetry operations

In [5]:
e = SymmetryOperation.from_vector( [ 1, 2, 3 ], label='E' )
e

SymmetryOperation
label(E)
matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

TODO: explain inversion of symmetry operations

In [6]:
c_3     = SymmetryOperation.from_vector( [ 2, 3, 1 ], label='C3' )
c_3_inv = SymmetryOperation.from_vector( [ 3, 1, 2 ], label='C3_inv' )

In [7]:
print( c_3 )
print( c_3_inv )

SymmetryOperation
label(C3)
matrix([[0, 0, 1],
        [1, 0, 0],
        [0, 1, 0]])
SymmetryOperation
label(C3_inv)
matrix([[0, 1, 0],
        [0, 0, 1],
        [1, 0, 0]])


Because `c_3_inv` is the inverse of `c_3` it can also be generated using the `.invert()` method

In [8]:
c_3.invert()

SymmetryOperation
label(---)
matrix([[0, 1, 0],
        [0, 0, 1],
        [1, 0, 0]])

In [9]:
c_3.invert( label= 'C3_inv')

SymmetryOperation
label(C3_inv)
matrix([[0, 1, 0],
        [0, 0, 1],
        [1, 0, 0]])

Note the resulting `SymmetryOperation` does not have a label defined. This can be set directly, or by chaining the `.set_label()` method, e.g.

In [10]:
c_3.invert().set_label( 'C3_inv' )

SymmetryOperation
label(C3_inv)
matrix([[0, 1, 0],
        [0, 0, 1],
        [1, 0, 0]])

Check that `c_3_inv` is the inverse of `c_3` (also demonstrate multiplication of `SymmetryOperation` objects).

In [11]:
c_3 * c_3.invert()

SymmetryOperation
label(---)
matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

In [12]:
( c_3 * c_3.invert() ).matrix == e.matrix

matrix([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]], dtype=bool)

In [13]:
# construct SymmetryOperations for C_3v group
# TODO add pictures for the full spacegroup
c_3_inv = c_3.invert( label='C3_inv' )
sigma_a = SymmetryOperation.from_vector( [ 1, 3, 2 ], label='S_a' )
sigma_b = SymmetryOperation.from_vector( [ 3, 2, 1 ], label='S_b' )
sigma_c = SymmetryOperation.from_vector( [ 2, 1, 3 ], label='S_c' )

## Symmetry Groups

In `bsym`, `SymmetryGroup` objects represent collections of `SymmetryOperation` objects. Note that a `SymmetryGroup` is not required to contain _all_ the symmetry operations of a particular **configuration space**, and therefore may not describe a complete mathematical <a href="https://en.wikipedia.org/wiki/Group_(mathematics)#Definition">group</a>.

For convenience `bsym` has `PointGroup` and `SpaceGroup` classes, that are equivalent to the `SymmetryGroup` parent class.

In [14]:
from bsym import PointGroup

In [15]:
c3v = PointGroup( [ e, c_3, c_3_inv, sigma_a, sigma_b, sigma_c ] )

In [16]:
c3v

PointGroup
E	[1, 2, 3]
C3	[3, 1, 2]
C3_inv	[2, 3, 1]
S_a	[1, 3, 2]
S_b	[3, 2, 1]
S_c	[2, 1, 3]

In [17]:
from bsym import ConfigurationSpace

In [18]:
c = ConfigurationSpace( objects=['a', 'b', 'c' ], symmetry_group=c3v )

In [19]:
c

ConfigurationSpace
['a', 'b', 'c']
E	[1, 2, 3]
C3	[3, 1, 2]
C3_inv	[2, 3, 1]
S_a	[1, 3, 2]
S_b	[3, 2, 1]
S_c	[2, 1, 3]

# Symmetry operation operating on a Configuration

In [20]:
from bsym import Configuration
c1 = Configuration.from_tuple( ( 1, 1, 0 ) )
c1

Configuration([1, 1, 0])

In [21]:
c_3 * c1

Configuration([0, 1, 1])

In [22]:
c_3.operate_on( c1 )

Configuration([0, 1, 1])

## Similarity Transforms

\begin{equation}
S^{-1}MS
\end{equation}

In [23]:
sigma_a

SymmetryOperation
label(S_a)
matrix([[1, 0, 0],
        [0, 0, 1],
        [0, 1, 0]])

In [24]:
c_3

SymmetryOperation
label(C3)
matrix([[0, 0, 1],
        [1, 0, 0],
        [0, 1, 0]])

In [25]:
sigma_a.similarity_transform( c_3 )

SymmetryOperation
label(---)
matrix([[0, 1, 0],
        [1, 0, 0],
        [0, 0, 1]])

In [26]:
sigma_c

SymmetryOperation
label(S_c)
matrix([[0, 1, 0],
        [1, 0, 0],
        [0, 0, 1]])

## Calculating all symmetry inequivalent permutations.

In [27]:
c

ConfigurationSpace
['a', 'b', 'c']
E	[1, 2, 3]
C3	[3, 1, 2]
C3_inv	[2, 3, 1]
S_a	[1, 3, 2]
S_b	[3, 2, 1]
S_c	[2, 1, 3]

In [28]:
c.unique_configurations( {1:1,0:1,2:1} )

[Configuration([0, 1, 2])]

Need to do this on a system where the result is not a single configuration.

In [29]:
c = ConfigurationSpace( [ 'a', 'b', 'c', 'd' ] ) # four vector configuration space

In [30]:
c 
# Initialisation of the ConfigurationSpace without a SymmetryGroup 
# defaults to only having the identity as a symmetry operation

ConfigurationSpace
['a', 'b', 'c', 'd']
E	[1, 2, 3, 4]

In [31]:
c.unique_configurations( {1:1, 0:3} )

[Configuration([0, 0, 0, 1]),
 Configuration([0, 0, 1, 0]),
 Configuration([0, 1, 0, 0]),
 Configuration([1, 0, 0, 0])]

In [32]:
c.symmetry_group.append( SymmetryOperation.from_vector( [ 4,3,2,1 ], label='sigma' ) )

SymmetryGroup
E	[1, 2, 3, 4]
sigma	[4, 3, 2, 1]

In [33]:
c.unique_configurations( {1:1, 0:3} )

[Configuration([0, 0, 0, 1]), Configuration([0, 0, 1, 0])]

In [34]:
c.unique_configurations( {1:1, 2:1, 0:2} )

[Configuration([0, 0, 1, 2]),
 Configuration([0, 0, 2, 1]),
 Configuration([0, 1, 0, 2]),
 Configuration([0, 1, 2, 0]),
 Configuration([0, 2, 0, 1]),
 Configuration([1, 0, 0, 2])]

In [35]:
# What if this represents points on a square?
c = ConfigurationSpace( [ 'a', 'b', 'c', 'd' ] ) # four vector configuration space
c.symmetry_group.append( SymmetryOperation.from_vector( [ 2, 3, 4, 1 ], label='C4' ) )
c.symmetry_group.append( SymmetryOperation.from_vector( [ 4, 1, 2, 3 ], label='C4i' ) )
c.symmetry_group.append( SymmetryOperation.from_vector( [ 4, 1, 2, 3 ], label='C2' ) )
c.symmetry_group.append( SymmetryOperation.from_vector( [ 4, 3, 2, 1 ], label='s_x' ) )
c.symmetry_group.append( SymmetryOperation.from_vector( [ 2, 1, 4, 3 ], label='s_y' ) )
c

ConfigurationSpace
['a', 'b', 'c', 'd']
E	[1, 2, 3, 4]
C4	[4, 1, 2, 3]
C4i	[2, 3, 4, 1]
C2	[2, 3, 4, 1]
s_x	[4, 3, 2, 1]
s_y	[2, 1, 4, 3]

In [36]:
c.unique_configurations( {1:2, 0:2} )

[Configuration([0, 0, 1, 1]), Configuration([0, 1, 0, 1])]